In [1]:
pip install sparqlwrapper

Note: you may need to restart the kernel to use updated packages.


Properties dont work, I limited result bcs of this

In [2]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

wikidata_url = "https://query.wikidata.org/sparql"
sparql = SPARQLWrapper(wikidata_url)

properties_query = """
SELECT DISTINCT ?property ?propertyLabel
WHERE {
  ?object ?property ?value . 
  ?object wdt:P1441|wdt:P361 ?game .
  ?game wdt:P31/wdt:P279* wd:Q7889 .
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
    ?property rdfs:label ?propertyLabel .
  }
}
Limit 2000
"""

sparql.setQuery(properties_query)
sparql.setReturnFormat(JSON)

properties_wikidata = sparql.query().convert()

In [3]:
properties_wikidata_l = []
for res in properties_wikidata['results']['bindings']:
    properties_wikidata_l.append({'property': res["property"]["value"], "propertyLabel": res['propertyLabel']['value']})

In [4]:
properties_df = pd.DataFrame(properties_wikidata_l)

In [5]:
properties_df

,property,propertyLabel
0,http://www.wikidata.org/prop/P279,http://www.wikidata.org/prop/P279
1,http://www.wikidata.org/prop/direct/P279,http://www.wikidata.org/prop/direct/P279
2,http://www.wikidata.org/prop/direct/P361,http://www.wikidata.org/prop/direct/P361
3,http://www.wikidata.org/prop/P361,http://www.wikidata.org/prop/P361
4,http://www.wikidata.org/prop/direct/P462,http://www.wikidata.org/prop/direct/P462
...,...,...
1995,http://www.wikidata.org/prop/direct/P9594,http://www.wikidata.org/prop/direct/P9594
1996,http://www.wikidata.org/prop/direct/P9778,http://www.wikidata.org/prop/direct/P9778
1997,http://www.wikidata.org/prop/direct/P9807,http://www.wikidata.org/prop/direct/P9807
1998,http://www.wikidata.org/prop/direct/P9885,http://www.wikidata.org/prop/direct/P9885


In [6]:
classes_query = """
SELECT DISTINCT ?class ?classLabel
WHERE {
  # Find all objects present in games (or subclasses of games)
  ?object wdt:P31 ?class .  # P31 represents "instance of"

  # Use "present in" or "part of" to capture context
  ?object wdt:P1441|wdt:P361 ?game .  # P1441 or P361 ("part of")

  # Ensure the game is an instance or subclass of "game"
  ?game wdt:P31/wdt:P279* wd:Q7889 .

  # Get labels for the object, property, and class
  SERVICE wikibase:label {
    bd:serviceParam wikibase:language "en" .
    ?class rdfs:label ?classLabel .
  }
}
"""
sparql.setQuery(classes_query)
sparql.setReturnFormat(JSON)

# Execute the query
classes_wikidata = sparql.query()

In [7]:
classes_wikidata = classes_wikidata.convert()

In [8]:
classes_wikidata_l = []
for res in classes_wikidata['results']['bindings']:
    classes_wikidata_l.append({'class': res["class"]["value"], "classLabel": res['classLabel']['value']})

In [9]:
classes_df = pd.DataFrame(classes_wikidata_l)

In [10]:
classes_df

,class,classLabel
0,http://www.wikidata.org/entity/Q1114461,comics character
1,http://www.wikidata.org/entity/Q1569167,video game character
2,http://www.wikidata.org/entity/Q3658341,literary character
3,http://www.wikidata.org/entity/Q15632617,fictional human
4,http://www.wikidata.org/entity/Q15773317,television character
...,...,...
1613,http://www.wikidata.org/entity/Q44180928,ship manufacturer
1614,http://www.wikidata.org/entity/Q312466,utility software
1615,http://www.wikidata.org/entity/Q65972034,video game trailer
1616,http://www.wikidata.org/entity/Q287618,sequential game


In [11]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

wikidata_url = "http://dbpedia.org/sparql"
sparql = SPARQLWrapper(wikidata_url)

properties_query = """
SELECT DISTINCT ?property ?propertyLabel
WHERE {
    ?object ?property ?value;
    dbo:wikiPageWikiLink ?game .
    ?game rdf:type dbo:VideoGame .

    OPTIONAL { ?property rdfs:label ?propertyLabel FILTER (lang(?propertyLabel) = 'en') }
}
"""

sparql.setQuery(properties_query)
sparql.setReturnFormat(JSON)

properties_dbpedia = sparql.query().convert()

In [12]:
properties_dbpedia_l = []
for res in properties_dbpedia['results']['bindings']:
    try:
        properties_dbpedia_l.append({'property': res["property"]["value"], "propertyLabel": res['propertyLabel']['value']})
    except:
        properties_dbpedia_l.append({'property': res["property"]["value"], "propertyLabel": None})

In [13]:
tmp = pd.DataFrame(properties_dbpedia_l)

In [14]:
properties_df = pd.concat([tmp, properties_df])

In [15]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

wikidata_url = "http://dbpedia.org/sparql"
sparql = SPARQLWrapper(wikidata_url)

classes_query = """
SELECT DISTINCT ?class ?classLabel
WHERE {
    ?object rdf:type ?class ;
    dbo:wikiPageWikiLink ?game .
    ?game rdf:type dbo:VideoGame .

    OPTIONAL { ?class rdfs:label ?classLabel FILTER (lang(?classLabel) = 'en') }
}
"""

sparql.setQuery(classes_query)
sparql.setReturnFormat(JSON)

classes_dbpedia = sparql.query().convert()

In [16]:
classes_dbpedia_l = []
for res in classes_dbpedia['results']['bindings']:
    try:
        classes_dbpedia_l.append({'class': res["class"]["value"], "classLabel": res['classLabel']['value']})
    except:
        classes_dbpedia_l.append({'class': res["class"]["value"], "classLabel": None})

In [17]:
tmp = pd.DataFrame(classes_dbpedia_l)

In [19]:
classes_df = pd.concat([tmp, classes_df])

In [20]:
classes_df

,class,classLabel
0,http://dbpedia.org/ontology/Company,company
1,http://dbpedia.org/ontology/Activity,activity
2,http://dbpedia.org/ontology/Person,person
3,http://dbpedia.org/ontology/Place,place
4,http://dbpedia.org/ontology/Software,software
...,...,...
1613,http://www.wikidata.org/entity/Q44180928,ship manufacturer
1614,http://www.wikidata.org/entity/Q312466,utility software
1615,http://www.wikidata.org/entity/Q65972034,video game trailer
1616,http://www.wikidata.org/entity/Q287618,sequential game


In [21]:
properties_df

,property,propertyLabel
0,http://dbpedia.org/property/name,name
1,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,None
2,http://www.w3.org/2000/01/rdf-schema#label,None
3,http://www.w3.org/2000/01/rdf-schema#comment,None
4,http://www.w3.org/2000/01/rdf-schema#seeAlso,None
...,...,...
1995,http://www.wikidata.org/prop/direct/P9594,http://www.wikidata.org/prop/direct/P9594
1996,http://www.wikidata.org/prop/direct/P9778,http://www.wikidata.org/prop/direct/P9778
1997,http://www.wikidata.org/prop/direct/P9807,http://www.wikidata.org/prop/direct/P9807
1998,http://www.wikidata.org/prop/direct/P9885,http://www.wikidata.org/prop/direct/P9885


In [23]:
classes_df.to_csv('RDF_classes.csv'), properties_df.to_csv('RDF_properties.csv')

(None, None)